In [5]:
import os
import json
import csv
from glob import glob

# Define the directory and file pattern
# path = "F:/setup files/downloads/Result2/"
path = "F:/setup files/downloads/ResultLastly/"
file_pattern = os.path.join(path, "*.json")

# List of files matching our pattern
files = glob(file_pattern)

# Prepare a list to store results
results = []

# Loop through each JSON file
for filename in files:
    # Load JSON data
    with open(filename, 'r') as f:
        data = json.load(f)

    # Determine number of objectives using the first algorithm/generation/solution
    # (Assuming at least one algorithm, one generation, and one individual exist)
    sample_alg = next(iter(data))
    sample_gen = next(iter(data[sample_alg]))
    sample_solution = data[sample_alg][sample_gen][0]
    objectives_count = len(sample_solution)

    # Initialize nadir point with very small values
    nadir = [-float('inf')] * objectives_count

    # Iterate over all algorithms, generations, and individuals to compute nadir
    for algorithm in data:
        for generation in range(1,100):
            # print(algorithm, generation)
            for individual in data[algorithm][str(generation)]:
                # Update nadir for each objective dimension
                for idx, value in enumerate(individual):
                    if value > nadir[idx]:
                        nadir[idx] = value

    # Store filename (or parsed info) and computed nadir point
    base_filename = os.path.basename(filename)
    result_row = {"file": base_filename}
    # Add each objective's nadir value to the row
    for j in range(objectives_count):
        result_row[f"obj_{j+1}_nadir"] = nadir[j]

    results.append(result_row)

# Define CSV fieldnames based on objectives count
fieldnames = ["file"] + [f"obj_{j+1}_nadir" for j in range(objectives_count)]

# Write results to a CSV file
output_csv = os.path.join(path, "nadir_points.csv")
with open(output_csv, 'w', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(results)

print(f"Nadir points saved to {output_csv}")

Nadir points saved to F:/setup files/downloads/CompareUtil/nadir_points.csv


In [1]:
import os
import json
import pandas as pd
from pymoo.indicators.hv import HV
import numpy as np

path = "F:/setup files/downloads/ResultLastly/"

def cal_hv(front, ref_point):
    with np.errstate(divide='ignore', invalid='ignore'):
        scaled_front = np.divide(front, ref_point, out=np.zeros_like(front), where=ref_point!=0)
    scaled_ref_point = np.ones_like(ref_point)
    hv_indicator = HV(ref_point=scaled_ref_point)
    return hv_indicator(scaled_front)

# Paths and file info
# path = "F:/setup files/downloads/Result2/"
nadir_csv_path = os.path.join(path, "nadir_points.csv")
hv_csv_path = os.path.join(path, "hv_values.csv")
pivot_csv_path = os.path.join(path, "hv_pivot.csv")  # For pivoted result

# Load the nadir points CSV
nadir_df = pd.read_csv(nadir_csv_path)
print("Nadir Points DataFrame:")
print(nadir_df.head())

hv_results = []

# Iterate over each row (each file) in the nadir DataFrame
for index, row in nadir_df.iterrows():
    file_name = row['file']
    file_path = os.path.join(path, file_name)

    # Check if JSON file exists
    if not os.path.isfile(file_path):
        print(f"File {file_name} not found. Skipping.")
        continue

    # Load JSON data
    with open(file_path, 'r') as f:
        data = json.load(f)

    # Extract the nadir point for scaling
    obj_columns = sorted(
        [col for col in nadir_df.columns if col.startswith("obj_") and col.endswith("_nadir")],
        key=lambda x: int(x.split('_')[1])
    )
    nadir_point = np.array([row[col] for col in obj_columns])

    # For each algorithm in the JSON data

    last = "100"
    for algorithm in data:
        if last not in data[algorithm]:
            print(f"Generation {last} not found for algorithm {algorithm} in file {file_name}. Skipping.")
            continue

        solutions = data[algorithm][last]
        solutions_np = np.array(solutions)

        if solutions_np.size == 0:
            print(f"No solutions found for algorithm {algorithm} in generation {last} of file {file_name}. Skipping.")
            continue

        try:
            hv_value = cal_hv(solutions_np, nadir_point)
        except Exception as e:
            print(f"Error computing HV for algorithm {algorithm} in file {file_name}: {e}")
            continue

        hv_results.append({
            "file": file_name,
            "algorithm": algorithm,
            "HV": hv_value
        })

# Convert HV results to a DataFrame
hv_df = pd.DataFrame(hv_results)

# Pivot the DataFrame: rows as file names, columns as algorithms, values as HV
hv_pivot_df = hv_df.pivot(index='file', columns='algorithm', values='HV')

# Save the pivoted HV DataFrame to a CSV file
hv_pivot_df.to_csv(pivot_csv_path)

print(f"Pivoted hypervolume table saved to {pivot_csv_path}")


Nadir Points DataFrame:
              file  obj_1_nadir  obj_2_nadir  obj_3_nadir  obj_4_nadir
0  LC1_2_10_0.json  1889.260877   191.618082  1921.153037  3868.706603
1   LC1_2_1_0.json  2004.677867   216.106415  3013.231277  5530.458606
2   LC1_2_2_0.json  1917.356225   186.458585  2646.831075  4881.059242
3   LC1_2_3_0.json  1797.444220   237.331890  2851.857745  5702.705454
4   LC1_2_4_0.json  1623.497047   136.508705  1849.709111  3399.629740
Pivoted hypervolume table saved to F:/setup files/downloads/ResultLastly/hv_pivot.csv
